## Backtracking
What: Systematic exploration of solution spaces by incrementally building candidates and abandoning ("backtracking") those that cannot lead to valid solutions.

When to use:
- Combinatorial problems: generating all permutations, combinations, subsets
- Constraint satisfaction: N-Queens, Sudoku solving, graph coloring
- Path finding: finding all paths in a maze or graph with certain properties

Key insight: Build solutions incrementally; as soon as you determine a partial candidate cannot lead to a valid solution, abandon it and backtrack.

Template:
```python
func backtrack(path []int, choices []int) {
    if isComplete(path) {
        processSolution(path)
        return
    }
    for _, choice := range getChoices(path) {
        if isValid(append(path, choice)) {
            path = append(path, choice)
            backtrack(path, remainingChoices)
            path = path[:len(path)-1]  // backtrack
        }
    }
}
```

Complexity: Exponential in worst case, but pruning can dramatically reduce the search space.

Edge cases: Empty input, constraints that make no solution possible, duplicate handling.

In [3]:
package main

import "fmt"

// 1) Subsets (power set)
func subsets(nums []int) [][]int {
    var res [][]int
    var path []int
    n := len(nums)

    var dfs func(int)
    dfs = func(i int) {
        if i == n {
            temp := make([]int, len(path))
            copy(temp, path)
            res = append(res, temp)
            return
        }
        // choice: include nums[i]
        path = append(path, nums[i])
        dfs(i + 1)
        path = path[:len(path)-1]
        // choice: exclude nums[i]
        dfs(i + 1)
    }

    dfs(0)
    return res
}

%%
fmt.Println("Subsets of [1,2,3]:", subsets([]int{1, 2, 3}))

Subsets of [1,2,3]: [[1 2 3] [1 2] [1 3] [1] [2 3] [2] [3] []]


In [4]:
// 2) Permutations
func permute(nums []int) [][]int {
    var res [][]int
    var path []int
    used := make([]bool, len(nums))

    var dfs func()
    dfs = func() {
        if len(path) == len(nums) {
            temp := make([]int, len(path))
            copy(temp, path)
            res = append(res, temp)
            return
        }
        for i, val := range nums {
            if used[i] {
                continue
            }
            used[i] = true
            path = append(path, val)
            dfs()
            path = path[:len(path)-1]
            used[i] = false
        }
    }

    dfs()
    return res
}

%%
perms := permute([]int{1, 2, 3})
maxShow := 3
if len(perms) < maxShow {
    maxShow = len(perms)
}
fmt.Println("Permutations of [1,2,3] (first 3):", perms[:maxShow])

Permutations of [1,2,3] (first 3): [[1 2 3] [1 3 2] [2 1 3]]


In [5]:
// 3) N-Queens
func solveNQueens(n int) [][]string {
    var res [][]string
    cols := make(map[int]bool)
    diag1 := make(map[int]bool) // r - c
    diag2 := make(map[int]bool) // r + c
    board := make([][]rune, n)
    for i := range board {
        board[i] = make([]rune, n)
        for j := range board[i] {
            board[i][j] = '.'
        }
    }

    var dfs func(int)
    dfs = func(r int) {
        if r == n {
            var solution []string
            for _, row := range board {
                solution = append(solution, string(row))
            }
            res = append(res, solution)
            return
        }
        for c := 0; c < n; c++ {
            if cols[c] || diag1[r-c] || diag2[r+c] {
                continue
            }
            cols[c] = true
            diag1[r-c] = true
            diag2[r+c] = true
            board[r][c] = 'Q'
            dfs(r + 1)
            board[r][c] = '.'
            delete(cols, c)
            delete(diag1, r-c)
            delete(diag2, r+c)
        }
    }

    dfs(0)
    return res
}

%%
fmt.Println("N-Queens n=4 solutions:")
for _, sol := range solveNQueens(4) {
    for _, row := range sol {
        fmt.Println(row)
    }
    fmt.Println()
}

N-Queens n=4 solutions:
.Q..
...Q
Q...
..Q.

..Q.
Q...
...Q
.Q..



In [6]:
import "sort"

// 4) Combination Sum (repetition allowed)
func combinationSum(candidates []int, target int) [][]int {
    sort.Ints(candidates)
    var res [][]int
    var path []int

    var dfs func(int, int)
    dfs = func(start, remain int) {
        if remain == 0 {
            temp := make([]int, len(path))
            copy(temp, path)
            res = append(res, temp)
            return
        }
        for i := start; i < len(candidates); i++ {
            c := candidates[i]
            if c > remain {
                break
            }
            path = append(path, c)
            dfs(i, remain-c) // i again allows reuse
            path = path[:len(path)-1]
        }
    }

    dfs(0, target)
    return res
}

%%
fmt.Println("Combination Sum for [2,3,6,7], target=7:", combinationSum([]int{2,3,6,7}, 7))

Combination Sum for [2,3,6,7], target=7: [[2 2 3] [7]]
